<a target="_blank" href="https://colab.research.google.com/github/lukebarousse/Int_SQL_Data_Analytics_Course/blob/main/6_Data_Cleaning/2_Strings.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# String Formatting

## Overview

### 🥅 Analysis Goals

- What we’re going to use for this dataset to do X e.g. Use the following in order to explore a dataset on experience and salaries
    - Major topic 1
    - Major topic 2
    - Major topic 3
- The end goal of this is e.g. Identify which jobs meet our expectations of years experience and total salary.

### 📘 Concepts Covered

General concepts we’re going to cover

- Concept 1
- Concept 2
- Concept 3

---
## CONCAT

### 📝 Notes

`CONCAT()`

- **CONCAT**: Combines two or more strings into a single string.

- Syntax:

  ```sql
  SELECT CONCAT(string1, string2, ...);
  ```

- Automatically handles `NULL` values as empty strings, avoiding `NULL` results when concatenating.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Go into specific step / what we’re going to do. E.g. Use the `=` operator to set a new column to be equal to Experience

---
## UPPER

### 📝 Notes

**`UPPER()`**

- **UPPER**: Converts a string to uppercase.

- Syntax:

  ```sql
  SELECT UPPER(string_column);
  ```

- Useful for standardizing text, such as converting names or codes to uppercase for comparison.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.

#### Problem Description

**`FUNCTION` / Concept Covered**

1. Go into specific step / what we’re going to do. E.g. Use the `=` operator to set a new column to be equal to Experience

---
## LOWER

### 📝 Notes

**`LOWER()`**

- **LOWER**: Converts a string to lowercase.  
- Syntax:  
  ```sql
  SELECT LOWER(string_column);
  ```
- Useful for standardizing text, such as making email addresses or usernames case-insensitive for comparisons.

### 💻 Final Result

- Describe what the final result should be e.g. return the retention by X cohort.



#### Problem Description

**`FUNCTION` / Concept Covered**

1. Go into specific step / what we’re going to do. E.g. Use the `=` operator to set a new column to be equal to Experience